<a href="https://colab.research.google.com/github/mgnarag/colored-binarization_lowres/blob/main/testing_nbs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls drive/My\ Drive
file_path = "/content/drive/My Drive/Architectural_designs/From Arki/"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf


from PIL import Image, ImageOps
Image.MAX_IMAGE_PIXELS = None
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd

from tensorflow import keras

import time
import os
import cv2

In [ ]:
def crop(im):
    width, height = im.size
    data = []
    for j in range(0,int(height/n_size)):
        for i in range(0,int(width/n_size)):
            im1 = im.crop((0 + (n_size*i), 0 + (n_size*j), n_size + (n_size*i), n_size + (n_size*j)))
            im1 = np.array(im1)
            im1 = im1.astype(np.float32)
            im1 = im1/255
            data.append(im1)
    return data


In [ ]:
gt = Image.open(file_path+ '4 Testing/ground_truth_100dpi.png').convert('L')
img_gt = cv2.imread(file_path+ '4 Testing/ground_truth_100dpi.png')
gt = np.asarray(gt) > 128
gt = ~gt

filename = "nbs_100dpi_wb.png"
directory = file_path + '4 Testing/' + filename
#directory = 'test_output_v'+str(model)+'.png'
image = Image.open(directory)
rgb = image.convert("RGB")
r, g, b = rgb.split()
w_dirty, h_dirty = r.size
dirty = [r,g,b]


METRIC = []
channel = ['red','green', 'blue']
for model in range(3,5):
  for color in range(0,3):
      print('Model = ', model, '', str(channel[color]))
      from tensorflow import keras
      autoencoder = keras.models.load_model(file_path + '1 Models/autoencoder_'+str(model)+ '_'+str(channel[color]))
      if model == 1:
          n_size = 32
      if model == 2:
          n_size = 32
      if model == 3:
          n_size = 64
      if model == 4:
          n_size = 64
      if model == 5:
          n_size = 128
      if model == 6:
          n_size = 128

      xx = int(w_dirty/n_size)
      final=[]

      for portion in range(0,xx):
          test = dirty[color]
          im1 = test.crop((n_size*portion, 0, (n_size*portion) + n_size, h_dirty))
          w1, h1 = im1.size
          w = int(w1/n_size)
          h = int(h1/n_size)

          neverbeforeseen = np.array(crop(im1))
          encoded_imgs = autoencoder.encoder(neverbeforeseen).numpy()
          decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

          col = np.vstack((decoded_imgs[0],decoded_imgs[1]))
          for i in range(2,h):
              col = np.vstack((col,decoded_imgs[i]))

          #y = np.where(col > 0.5,1,0) #round the values
          y = col
          y = (y * 255).astype('uint8')
          if portion == 0:
              final = y
          if portion > 0:
              final = np.hstack((final,y))

      final = np.squeeze(final)
      reconstructed = Image.fromarray(final)
      reconstructed.save(file_path + "5 Tested/output v" + str(model) + '_'+str(channel[color]) + '.png')


  #COMBINING RGB ------------------------------------------------
  #MASKING EACH RGB ------------------------------------------------
  print('Model = ', model, '', str('RGB'))
  rgbArray = np.zeros((h_dirty,w_dirty,3), 'uint8')
  rgbArray[..., 0] = 1-np.asarray(Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[0]) +  '.png').convert('L'))
  rgbArray[..., 1] = 1-np.asarray(Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[1]) +  '.png').convert('L'))
  rgbArray[..., 2] = 1-np.asarray(Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[2]) +  '.png').convert('L'))

  final_RGB = Image.fromarray(rgbArray)
  final_RGB.save(file_path + "5 Tested/output v" + str(model) + '_RGB_colored.png')

Model =  1  red
Model =  1  green
Model =  1  blue
Model =  1  RGB
Model =  2  red
Model =  2  green
Model =  2  blue
Model =  2  RGB
Model =  3  red
Model =  3  green
Model =  3  blue
Model =  3  RGB
Model =  4  red
Model =  4  green
Model =  4  blue
Model =  4  RGB
Model =  5  red
Model =  5  green
Model =  5  blue
Model =  5  RGB
Model =  6  red
Model =  6  green
Model =  6  blue
Model =  6  RGB
